In [10]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
 
spark=SparkSession.builder.master('local').\
            config('spark.jars.packages','mysql:mysql-connector-java:5.1.44').\
            appName('db').getOrCreate()
  

#read table data into a spark dataframe
products = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/amu") \
    .option("dbtable", 'items') \
    .option("user", 'root') \
    .option("password", 'amruta') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .load()
 
#show the data loaded into dataframe
products.show()

+----+-----------+--------+
|prid|compna_name|category|
+----+-----------+--------+
| 111|     orange|     top|
| 112|     orange|     top|
| 113|      focus|   jeans|
| 114|      focus|   jeans|
| 115|      bloom|   jeans|
| 116|     infeel|  jacket|
| 117|     infeel|     top|
| 118|   london81|   jeans|
| 119|  vero moda|  jacket|
| 120|  vero moda|     top|
+----+-----------+--------+



In [11]:
history = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/amu") \
    .option("dbtable", 'purchase_history') \
    .option("user", 'root') \
    .option("password", 'amruta') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .load()
 
#show the data loaded into dataframe
history.show()

+---+----+
|uid|prid|
+---+----+
|101| 111|
|101| 113|
|101| 114|
|101| 119|
|102| 117|
|102| 120|
|102| 116|
|102| 118|
+---+----+



In [12]:
uid=int(input('id'))
user_bought_products=history.filter(history['uid']==uid)
user_bought_products.show()

id101
+---+----+
|uid|prid|
+---+----+
|101| 111|
|101| 113|
|101| 114|
|101| 119|
+---+----+



In [14]:
user_prod_cat =products.join(user_bought_products, products['prid'] == user_bought_products['prid'],how='inner') # Could also use 'left_outer'
user_prod_cat.show()

+----+-----------+--------+---+----+
|prid|compna_name|category|uid|prid|
+----+-----------+--------+---+----+
| 111|     orange|     top|101| 111|
| 114|      focus|   jeans|101| 114|
| 113|      focus|   jeans|101| 113|
| 119|  vero moda|  jacket|101| 119|
+----+-----------+--------+---+----+



In [21]:
max_cat=user_prod_cat.groupBy('category').count().orderBy('count',ascending=False).first()
max_cat

Row(category='jeans', count=2)

In [23]:
print(max_cat['category'])

jeans


In [25]:
all_cat_products=products.filter(products['category']==(max_cat['category']))
all_cat_products.show()                             

+----+-----------+--------+
|prid|compna_name|category|
+----+-----------+--------+
| 113|      focus|   jeans|
| 114|      focus|   jeans|
| 115|      bloom|   jeans|
| 118|   london81|   jeans|
+----+-----------+--------+



In [29]:
recc =all_cat_products.join(user_bought_products, on=['prid'], how='left_anti')
recc.show()

+----+-----------+--------+
|prid|compna_name|category|
+----+-----------+--------+
| 115|      bloom|   jeans|
| 118|   london81|   jeans|
+----+-----------+--------+

